In [140]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [164]:
URL = "https://www.englishclub.com/esl-quizzes/vocabulary/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [166]:
soup.find_all('li')

[<li><a href="fixed-expressions-quiz-1.php">Fixed Expressions Quiz 1</a></li>,
 <li><a href="fixed-expressions-quiz-2.php">Fixed Expressions Quiz 2</a></li>,
 <li><a href="fixed-expressions-quiz-3.php">Fixed Expressions Quiz 3</a></li>,
 <li><a href="fixed-expressions-quiz-4.php">Fixed Expressions Quiz 4</a></li>,
 <li><a href="fixed-expressions-quiz-5.php">Fixed Expressions Quiz 5</a></li>,
 <li><a href="fixed-expressions-quiz-6.php">Fixed Expressions Quiz 6</a></li>,
 <li><a href="fixed-expressions-quiz-7.php">Fixed Expressions Quiz 7</a></li>,
 <li><a href="fixed-expressions-quiz-8.php">Fixed Expressions Quiz 8</a></li>,
 <li><a href="fixed-expressions-quiz-9.php">Fixed Expressions Quiz 9</a></li>,
 <li><a href="fixed-expressions-quiz-10.php">Fixed Expressions Quiz 10</a></li>,
 <li><a href="/vocabulary/animal-terms-easy-quiz.php">Animal Terms (short list) Quiz</a></li>,
 <li><a href="/vocabulary/animal-terms-quiz.php">Animal Vocabulary Quiz</a></li>,
 <li><a href="/vocabulary/festi

In [179]:
a = soup.find_all('li')[1].find('a').get('href')

In [180]:
URL+a

'https://www.englishclub.com/esl-quizzes/vocabulary/fixed-expressions-quiz-2.php'

In [141]:
URL = 'https://www.englishclub.com/vocabulary/animal-terms-easy-quiz.php'
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [142]:
questions = soup.find_all("form", class_="mcquiz")[0]
choices = questions.find_all("input")

In [144]:
list_qno = []
list_questions = []
count_q = 1
for q in questions.find_all("p")[:len(list_answers)]:
    pattern = r'^\d+\.\s*'
    str_wo_num = re.sub(pattern, '', q.text)
    list_questions.append(str_wo_num)
    list_qno.append(count_q)
    count_q = count_q+1
    
df_questions = pd.DataFrame({'question_no':list_qno, 'question':list_questions})
# df_questions

,question_no,question
0,1,A male chicken is called
1,2,A young sheep is called
2,3,A female pig is called
3,4,A baby kangaroo is called
4,5,A tigress is a _______ tiger.
5,6,A foal is a _______ horse.
6,7,A vixen is a _______ fox.
7,8,Which word is used for a male elephant and a m...
8,9,Which word is used for a female dog and a fema...
9,10,"Which word is used for the young of wolves, ti..."


In [145]:
# extract answer
answer_str = soup.find_all('script', {"type": "text/javascript", "language":"JavaScript"})[0].text
pattern = r'answers\[\d\]="([^"]+)"'
matches = re.findall(pattern, answer_str)

list_qno = []
list_answers = []
count_q = 1
for match in matches:
    list_answers.append(match)
    list_qno.append(count_q)
    count_q = count_q+1
df_answers = pd.DataFrame({'question_no':list_qno, 'answer':list_answers})
# df_answers

,question_no,answer
0,1,a rooster
1,2,a lamb
2,3,a sow
3,4,a joey
4,5,female
5,6,young
6,7,female
7,8,bull
8,9,bitch
9,10,cub


In [146]:
list_choices_qno = []
list_choices = []
list_choices_no = []
for i in range(1, len(list_answers)+1):
   
    count_c = 1
    for c in choices[:-3]:
#         print(c)
        try:
            if int(c.get('name')[1:])==i:
                list_choices_qno.append(i)
                list_choices_no.append(count_c)
                list_choices.append(c.get('value'))
                count_c = count_c+1
        except:
            pass


In [147]:
df_choice = pd.DataFrame({'question_no':list_choices_qno, 'choices_no':list_choices_no, 'choices':list_choices})

In [148]:
# df_choice.head()

,question_no,choices_no,choices
0,1,1,a hen
1,1,2,a chick
2,1,3,a rooster
3,2,1,a ram
4,2,2,a lamb


In [149]:
df_quiz = pd.merge(df_questions, df_answers, on='question_no')
df_quiz = df_quiz.merge(df_choice, on='question_no')
# df_quiz.shape

(30, 5)

In [151]:
# df_quiz.head()

,question_no,question,answer,choices_no,choices
0,1,A male chicken is called,a rooster,1,a hen
1,1,A male chicken is called,a rooster,2,a chick
2,1,A male chicken is called,a rooster,3,a rooster
3,2,A young sheep is called,a lamb,1,a ram
4,2,A young sheep is called,a lamb,2,a lamb


In [162]:
q_grouped_dict = {group: group_df.drop('question_no', axis=1).to_dict(orient='records') 
                for group, group_df in df_quiz.groupby('question_no')}

print(q_grouped_dict)

{1: [{'question': 'A male chicken is called', 'answer': 'a rooster', 'choices_no': 1, 'choices': 'a hen'}, {'question': 'A male chicken is called', 'answer': 'a rooster', 'choices_no': 2, 'choices': 'a chick'}, {'question': 'A male chicken is called', 'answer': 'a rooster', 'choices_no': 3, 'choices': 'a rooster'}], 2: [{'question': 'A young sheep is called ', 'answer': 'a lamb', 'choices_no': 1, 'choices': 'a ram'}, {'question': 'A young sheep is called ', 'answer': 'a lamb', 'choices_no': 2, 'choices': 'a lamb'}, {'question': 'A young sheep is called ', 'answer': 'a lamb', 'choices_no': 3, 'choices': 'a ewe'}], 3: [{'question': 'A female pig is called ', 'answer': 'a sow', 'choices_no': 1, 'choices': 'a sow'}, {'question': 'A female pig is called ', 'answer': 'a sow', 'choices_no': 2, 'choices': 'a suckling'}, {'question': 'A female pig is called ', 'answer': 'a sow', 'choices_no': 3, 'choices': 'a boar'}], 4: [{'question': 'A baby kangaroo is called', 'answer': 'a joey', 'choices_no

In [163]:
json_string = json.dumps(q_grouped_dict)

print(json_string)

{"1": [{"question": "A male chicken is called", "answer": "a rooster", "choices_no": 1, "choices": "a hen"}, {"question": "A male chicken is called", "answer": "a rooster", "choices_no": 2, "choices": "a chick"}, {"question": "A male chicken is called", "answer": "a rooster", "choices_no": 3, "choices": "a rooster"}], "2": [{"question": "A young sheep is called ", "answer": "a lamb", "choices_no": 1, "choices": "a ram"}, {"question": "A young sheep is called ", "answer": "a lamb", "choices_no": 2, "choices": "a lamb"}, {"question": "A young sheep is called ", "answer": "a lamb", "choices_no": 3, "choices": "a ewe"}], "3": [{"question": "A female pig is called ", "answer": "a sow", "choices_no": 1, "choices": "a sow"}, {"question": "A female pig is called ", "answer": "a sow", "choices_no": 2, "choices": "a suckling"}, {"question": "A female pig is called ", "answer": "a sow", "choices_no": 3, "choices": "a boar"}], "4": [{"question": "A baby kangaroo is called", "answer": "a joey", "ch